In [1]:
import cv2
import torch
import os
model = torch.hub.load('ultralytics/yolov5', 'yolov5s',verbose=False)

def detect_objects(frame_path, model, output_dir, conf_threshold=0.5, iou_threshold=0.45):
    

    if not os.path.exists(frame_path):
        raise FileNotFoundError(f"Frame file not found: {frame_path}")

    img = cv2.imread(frame_path)
    results = model(img)

    # Process results (with class filtering)
    allowed_classes = [2, 3, 5, 7, 0, 9, 11, 1] 
    for pred in results.pred[0]:
        conf = pred[-1]
        if conf > conf_threshold:
            class_id = int(pred[5])
            if class_id not in allowed_classes:
                continue

            bbox = pred[:4].detach().cpu().numpy().astype(int)
            x1, y1, x2, y2 = bbox

            # Get class label and confidence
            label = model.names[class_id]
            score = round(conf.item(), 2)

            # Draw bounding box and label
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label_text = f"{label}: {score}"
            cv2.putText(img, label_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Save annotated frame
    output_path = os.path.join(output_dir, os.path.basename(frame_path))
    cv2.imwrite(output_path, img)

if __name__ == "__main__":

    frames_dir = "extracted_frames/"
    output_dir = "annotated_frames/"  
    os.makedirs(output_dir, exist_ok=True)

    for filename in os.listdir(frames_dir):
        if filename.endswith(".jpg"):
            frame_path = os.path.join(frames_dir, filename)
            detect_objects(frame_path, model, output_dir)



YOLOv5  2024-7-1 Python-3.11.7 torch-2.3.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
